<a href="https://colab.research.google.com/github/Faraz-Khan02/Book-Recommendation-System/blob/main/Book_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - **Book Recommendation System**



##### **Project Type**    - Unsupervised
##### **Contribution**    - Individual
##### **Name**   -     Faraz Faisal Khan


# **Project Summary -**

Recommendation system is one of the evolving field nowadays, from e-commerce to online advertisement recommendation system is used everywhere. First of all for doing this Unsupervised learning we 

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


**During the last few decades, with the rise of Youtube, Amazon, Netflix, and many other such web services, recommender systems have taken more and more place in our lives. From e-commerce (suggest to buyers articles that could interest them) to online advertisement (suggest to users the right contents, matching their preferences), recommender systems are today unavoidable in our daily online journeys.
In a very general way, recommender systems are algorithms aimed at suggesting relevant items to users (items being movies to watch, text to read, products to buy, or anything else depending on industries).
Recommender systems are really critical in some industries as they can generate a huge amount of income when they are efficient or also be a way to stand out significantly from competitors. The main objective is to create a book recommendation system for users.**

# ***Let's Begin !***

### Import Libraries

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import math
import random
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings("ignore")

### Dataset Loading

In [ ]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load Dataset
books = pd.read_csv("/content/drive/MyDrive/Capstone Project - 4/Books.csv")
users = pd.read_csv("/content/drive/MyDrive/Capstone Project - 4/Users.csv")
ratings = pd.read_csv("/content/drive/MyDrive/Capstone Project - 4/Ratings.csv")

## **Books Data**

In [ ]:
# Books data first look
books.head()

Here, Books data contains many features which are as follows:


*   **ISBN** : It contains ISBN number of the books which mean  International Standard Book Number.
*   **Book-Title** : It contains title of the books.


*   **Book-Author** : It contains the name of author of that book.
*   **Year-Of-Publication** : It contains the year in which that book was published.


*   **Publisher** : It contains the name of the publisher.
*   **Image-URL-S** : It contains the Image Url of small size.


*   **Image-URL-M** : It contains the Image Url of medium size.
*   **Image-URL-L** : It contains the Image Url of Large size.









In [ ]:
# Shape of Books data
books.shape

Our, Books dataset contains 271360 rows and 8 columns.

In [ ]:
# Books dataset info
books.info

In [ ]:
# Books Dataset Duplicate Value Count
duplicate = books.duplicated()
print(duplicate.value_counts())

It means there is no any duplicate values in the books dataset.

In [ ]:
# Missing Values/Null Values Count
books.isnull().sum()

## ***Data Cleaning of Books Dataset***

Here, Image-URL-S, Image-URL-M, Image-URL-L are not any useful feature for our recommmendation so we will drop it.

In [ ]:
# dropping last three columns 
books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'],axis=1,inplace=True)

In [ ]:
# Renaming the Books column
books.rename(columns = {'Book-Title':'Book_Title', 'Book-Author':'Book_Author', 'Year-Of-Publication':'Year_Of_Publication'}, inplace=True)

Changing the columns name because dashed sign caused some problem further.

In [ ]:
# After dropping 3 columns
books.head()

In [ ]:
# Checking data types of columns
print(books.dtypes)

Here, Year_Of_Publication should have integer datatype but its object so we will check the unique values of it.

In [ ]:
#Get the unique values of Year-Of-Publication	
books['Year_Of_Publication'].unique()

Here we can see that 'DK Publishing Inc' and 'Gallimard' are wrong entry and we can see Year of Publication is more than 2004 which is wrong because our data was published in 2004.

In [ ]:
#Checking the rows having 'DK Publishing Inc' and 'Gallimard' as Year_Of_Publication
books.loc[(books['Year_Of_Publication'] == 'DK Publishing Inc') |( books['Year_Of_Publication'] == 'Gallimard'),:]

Here, we can clearly see that Book-Author and Year_Of_Publication is mismatched so we will replace it correctly.

In [ ]:
# Correcting 1st row 
books.loc[books.ISBN == '0789466953','Year_Of_Publication'] = 2000
books.loc[books.ISBN == '0789466953','Book_Author'] = "James Buckley"
books.loc[books.ISBN == '0789466953','Publisher'] = "DK Publishing Inc"
books.loc[books.ISBN == '0789466953','Book_Title'] = "DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)"

In [ ]:
# Correcting 2nd row
books.loc[books.ISBN == '2070426769','Year_Of_Publication'] = 2003
books.loc[books.ISBN == '2070426769','Book_Author'] = "Jean-Marie Gustave Le ClÃ?Â©zio"
books.loc[books.ISBN == '2070426769','Publisher'] = "Gallimard"
books.loc[books.ISBN == '2070426769','Book_Title'] = "Peuple du ciel, suivi de 'Les Bergers"

In [ ]:
# Correcting 3rd row
books.loc[books.ISBN == '078946697X','Year_Of_Publication'] = 2000
books.loc[books.ISBN == '078946697X','Book_Author'] = "Michael Teitelbaum"
books.loc[books.ISBN == '078946697X','Publisher'] = "DK Publishing Inc"
books.loc[books.ISBN == '078946697X','Book_Title'] = "DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)"

In [ ]:
#Rechecking after correcting
books.loc[(books.ISBN == '0789466953') | (books.ISBN == '078946697X') | (books.ISBN == '2070426769'),:]

In [ ]:
# Converting year of publication to type int
books['Year_Of_Publication'] = books['Year_Of_Publication'].astype(int)

In [ ]:
# Checking year in sorted manner
sorted(books['Year_Of_Publication'].unique())

Here, 0 is incorrect and our data was of 2004 and Year_Of_Publication is greater than 2004 which means it is wrongly interpreted.So, we will interpret those values with NaN. 

In [ ]:
# Removing year 0 and years above 2004 with NaN
books.loc[(books['Year_Of_Publication'] > 2004) | (books['Year_Of_Publication']==0),'Year_Of_Publication'] = np.NAN

## ***Handling Missing Values***

In [ ]:
#Checking for missing values
books.isnull().sum()

In [ ]:
# box plot for Year-Of-Piblication
plt.figure(figsize=(8,4))
sns.boxplot(books['Year_Of_Publication'])

Here, we can see Book Author contains '1' missing values , Year_Of_Publication contain 4690 missing values and Publisher contains '2' missing values. So, we will handle those missing values.

In [ ]:
#Imputing the NaN values with medain values of Year_Of_Publication
books['Year_Of_Publication'].fillna(round(books['Year_Of_Publication'].median()),inplace=True)

In [ ]:
# Publisher column has 2 NaN so exploring it
books.loc[books.Publisher.isnull(),:]

Here, we can can fill NaN values with others.

In [ ]:
# Replacing missing values with 'other'
books.loc[(books.ISBN == '193169656X'),'Publisher'] = 'other'
books.loc[(books.ISBN == '1931696993'),'Publisher'] = 'other'

Since, author has only 1 missing value so, we can drop that row easily.

In [ ]:
# Dropping the missing values in 'author' column 
books.dropna(axis=0,inplace=True)

In [ ]:
#Checking for missing values
books.isnull().sum()

So, here we can clearly see that we dont have any missing values in our Books dataset.

## **Users Data**

In [ ]:
# Users data first look
users.head()

Our Users dataset contains following features and they are:


*   **User-ID** : It contains the User-Id of different Users.
*   **Location** : It contains the location of the Users.

*   **Age** : It contains the age of the Users






In [ ]:
# Shape of the Users dataset
users.shape

Our data contains 278858 rows and 3 columns.

In [ ]:
# Users dataset info
users.info

In [ ]:
# Users Dataset Duplicate Value Count
duplicate = users.duplicated()
print(duplicate.value_counts())

so, here there is no any duplicate value in our dataset.

In [ ]:
# Missing Values/Null Values Count
users.isnull().sum()

Age has 110762 missing values which is a great number.

## ***Data Cleaning of Users Data***

In [ ]:
users.rename(columns = {'User-ID':'User_ID'}, inplace=True)

In [ ]:
# box plot for Age
plt.figure(figsize=(8,4))
sns.boxplot(users['Age'])

Here, we can clearly see Age column contains lot of outliers.

In [ ]:
# Getting unique age values in sorted manner
print(sorted(users.Age.unique()))

Here, Age contains NaN values, 0 to 5 years and age greater than 90 no any user would be there who would read books. so will remove age less than 5 and age greater than 90 with NaN.

In [ ]:
# Replacing age below 5 and above 90 by NaN
users.loc[(users.Age > 90) | (users.Age < 5), 'Age'] = np.nan

In [ ]:
#Checking for missing values
users.isnull().sum()

So, here we will impute the missing values with mean.

In [ ]:
# Replacing NaN with mean
users['Age'].fillna((users['Age'].mean()), inplace=True)

In [ ]:
users.head()

So, here we will change the data type of Age to int.

In [ ]:
# setting the data type as int
users.Age = users.Age.astype(np.int64)

In [ ]:
#Checking for missing values
books.isnull().sum()

So, we have cleaned our Users Dataset.

## **Ratings Data**

In [ ]:
#Ratings Data first look
ratings.head()

Out Ratings Dataset contains following features:


*   **User-ID** : It contains the User-Id of different users.
*   **ISBN** : It contains ISBN number of the books which mean International Standard Book Number.

*   **Book-Rating** : It contains the rating of the book given by different users. 



In [ ]:
#Shape of the Ratings Dataset
ratings.shape

Our dataset contains 1149780 rows and 3 columns.

In [ ]:
# Ratings dataset info
ratings.info

In [ ]:
# Changing the name of the column
ratings.rename(columns = {'User-ID':'User_ID', 'Book-Rating':'Book_Rating'}, inplace=True)

In [ ]:
# Ratings Dataset Duplicate Value Count
duplicate = ratings.duplicated()
print(duplicate.value_counts())

so, there are no any duplicate value present in our dataset.

In [ ]:
# Missing Values/Null Values Count
ratings.isnull().sum()

So, here we can clearly see there is no any missing value in ratings. But ratings data contain many ISBN so, we will check it from books dataset.

In [ ]:
# we are agregating only unique ISBN from ratings which is in books dataset
unique_ratings = ratings[ratings.ISBN.isin(books.ISBN)]

Ratings dataset should have ratings from users which exist in users dataset, unless new users or book are added to users dataset.

In [ ]:
# Creating dataframe which contains books that are only in books dataset
ratings = ratings[ratings.User_ID.isin(users.User_ID)]

# **Exploratory Data Analysis**

## ***EDA on Books Data***

## Top 10 Books

In [ ]:
# Top 10 Books
ax = books['Book_Title'].value_counts()[:10].plot(kind = 'barh', legend = False, figsize = (12,6))
plt.title('Top 10 books')

From above visualization we infer that **Great Expectations**, **Pride and Prejudice**, **The Night Before Christmas**, **Jane Eyre**, **Adventures of Huckleberry Finn**, **The Secret Garden**, **Dracula**, **Wuthering heights**, **Little Women** and **Selected Poems** are the top 10 books in our dataset.

## Top 10 Authors

In [ ]:
# Visualizing top 10 authors
plt.figure(figsize=(12,8))
sns.countplot(y='Book_Author',data=books,order=pd.value_counts(books['Book_Author']).iloc[:10].index)
plt.title('Top 10 Authors')

From our Countplot we can infer that **Agatha Christie**, **William Shakespeare**, **Stephen King**, **Ann M Martin**, **Carolyn Keene**, **Francine Pascal**, **Isaac Asimov**, **Nora Roberts**, **Barbara Cartiand** and **Charles Dickens** are the top Authors.

## Top 10 Publishers

In [ ]:
# Visualizing top 10 Publisher
plt.figure(figsize=(12,8))
sns.countplot(y='Publisher',data=books,order=pd.value_counts(books['Publisher']).iloc[:10].index)
plt.title('Top 10 Publishers')

From our countplot we infer that **Harlequin**, **Silhouette**, **Pocket**, **Ballantine Books**, **Bantam Books**, **Scholastic**, **Simon &amp Schuster**, **Penguin Books**, **Berkley Publishing Group** and **Warner Books** are the top 10 publishers.

## Books Published in that Year

In [ ]:
# Visualizing the no. of books published each year through histogram
sns.set_style('darkgrid')
fig, ax =plt.subplots()
fig.set_size_inches(12,8)
sns.histplot(books['Year_Of_Publication'],bins=np.arange(1900,2004,3),color='r')
plt.ylabel('No. of Books Published')
plt.xlabel('Year')
plt.title('Visualizing the total no. of books published each year')
plt.show()

From our visualization we can say that in 2000 most no. of books was Published.

## ***EDA on Users Data***

## Which Age category reads the book most?

In [ ]:
#Plotting histogram of age distribution
fig = plt.figure(figsize = (12,8))
users.Age.hist(bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
plt.title('Age Distribution\n')
plt.xlabel('Age')
plt.ylabel('count')
plt.show()

From our histogram plotting we can see that mpst of the Users are from 30 - 40.

In [ ]:
#Plotting pie chart for above graph
fig = plt.figure(figsize = (12,12))
users.Age.value_counts().plot.pie(autopct='%1.1f%%',shadow=True)
plt.title('Age of Users')



From this we can clearly see 41.9% of users are age 34 this is becuase we have imputed mean value in it.

## Top 10 cities for Maximum Readers

In [ ]:
#Top 10 location where most no. of users recide
plt.figure(figsize=(12,8))
sns.countplot(x="Location", data=users,order=users['Location'].value_counts().index[0:10])
plt.title("Top 10 cities for maximum readers")

## **EDA on Ratings Data**

From above plot we infer that **london, england, united kingdom** is place where most number of users recide.

In [ ]:
# Plotting count of rating to see how it's distributed
fig = plt.figure(figsize = (12,6))
sns.countplot(x='Book_Rating',data=ratings)
plt.title("Rating plot")

Here, we can see 0 is the most in our rating dataset which means most of the users did not give any review. Now we have to separate the explicit ratings represented by 1–10 and implicit ratings represented by 0.

In [ ]:
# Separating ratings
ratings_explicit= unique_ratings[unique_ratings['Book_Rating'] != 0]
ratings_implicit= unique_ratings[unique_ratings['Book_Rating'] == 0]

In [ ]:
# Plotting count of rating to see how it's distributed
fig = plt.figure(figsize = (12,6))
sns.countplot(x='Book_Rating',data=ratings_explicit)
plt.title("Rating plot")

From this visualization we can see 8 is the most common rating given by the users and most of the ratings lie between 7 - 10.

# **Popularity Based Recommendation System**

Popularity based recommendation system works with the current vogue. It basically uses the items which are in swing at present. This is the most basic recommendation system which provides generalized recommendation to every user depending on the popularity. Whatever is more popular among the general public that is more likely to be recommended to new customers. The generalized recommendation not personalized is based on the count.

In [ ]:
#Viewing our explicit dataset
ratings_explicit.head()

In [ ]:
# Merging ratings_explicit with books dataset on ISBN feature
new_book= pd.merge(books, ratings_explicit, on='ISBN')
new_book.head()

Since, Book and Users data have both ISBN as common feauture so, we explicitily merged both the datasets.

In [ ]:
# Shape of new_book data
new_book.shape

Now, here new_book data contains 383841 rows and 7 columns. 

In [ ]:
# Grouping new_book data based on title and aggregate based on rating
top_ten_books= pd.DataFrame(new_book.groupby('Book_Title')['Book_Rating'].count().sort_values(ascending=False).head(10))

In [ ]:
# Printing top ten books 
print('The top ten books recommendations are :')
top_ten_books

We, can see clearly our top 10 books acoording to rating of the users. Then also we will see it by visualizing with a plot.

In [ ]:
# Reset the index  
top_ten_books=top_ten_books.reset_index() 

In [ ]:
# Plotting horizontal bar blot
plt.figure(figsize=(12, 8))
g=sns.barplot(x='Book_Rating',y='Book_Title',data=top_ten_books, palette="Oranges")
plt.title("Top Ten Books")

From our Recommendation System we infer that following books are mostly recommended books:

1.   The Lovely Bones: A Novel 
2.   Wild Animus

1.   The Da Vinci Code
2.   The Secret Life of Bees

1.   The Nanny Diaries: A Novel
2.   The Red Tent(Bestselling Backlist)

1.   Bridget Jones Diary
2.   A Painted House

1.   Life of Pie
2.   Harry Potter and the Chamber of Secrets( Book 2)














# **Collaborative Filtering using KNN**

This filtering method is usually based on collecting and analyzing information on user’s behaviors, their activities or preferences, and predicting what they will like based on the similarity with other users. A key advantage of the collaborative filtering approach is that it does not rely on machine analyzable content and thus it is capable of accurately recommending complex items such as movies without requiring an “understanding” of the item itself.


Here we assume that users who given ratings more than 200 are users who read atleast 20 books.For statisfical significance we should consider only the data of user who given more than 200 ratings.

In [ ]:
#Users with more than 200 ratings 
numbers1 = ratings_explicit['User_ID'].value_counts()
ratings = ratings_explicit[ratings_explicit['User_ID'].isin(numbers1[numbers1 >= 200].index)]
#Books with more than 100 Ratings
number2 = ratings_explicit['Book_Rating'].value_counts()
ratings = ratings_explicit[ratings_explicit['Book_Rating'].isin(number2[number2 >= 100].index)]

Here, I have taken rating >= 200 and excluded rating <= 100.

In [ ]:
#Merging the dataset ratings and Books
books_with_rating = pd.merge(ratings, books, on='ISBN')
books_with_rating.head()

Here, we have merged book and ratings data with the help of ISBN which are in common.

In [ ]:
# Grouping it with Book-Title and Book-Ratings  
rating_count_df =books_with_rating.groupby('Book_Title')['Book_Rating'].count().reset_index()

Here, I grouped the Book-Title and Book-rating because we need to predict the books on the basis of ratings.

In [ ]:
# the rating_count_df contain
rating_count_df.head()

Here, it only contain Book_Title and Book_Rating which we need in our Recommendation System.

In [ ]:
#Renaming the column Book-rating with Total-Rating
rating_count_df.rename(columns={'Book_Rating':'Total_Ratings'},inplace=True)

In [ ]:
#Merging the dataframes
total_Rating_Count = books_with_rating.merge(rating_count_df, left_on = 'Book_Title', right_on = 'Book_Title', how = 'left')
total_Rating_Count.head()

Here, we have merged two datasets books_with_rating and rating_count_df to know the popular books.

Now we wil keep only those books which are more popular according to total_book_rating.


In [ ]:
#Keeping threshold value 50 
popularity_threshold = 50
rating_popular_book = total_Rating_Count.query('Total_Ratings >= @popularity_threshold')
rating_popular_book.head()

Here, two daatframes are merged now its time to merge users dataframe to our rating_popular_book.

In [ ]:
# Merging all the three data sets
merged_data = rating_popular_book.merge(users, left_on = 'User_ID', right_on = 'User_ID', how = 'left')

In [ ]:
# Merged data contain
merged_data.head()

Here, we can see that our merged_data contains all  the necessary columns which is required for our recommendation sytem. The following features are User_ID, ISBN, Book_Rating, Book_Title, Book_Author, Year_Of_Publication, Publisher, Total_Ratings, Location and Age.

In [ ]:

# Ratings Dataset Duplicate Value Count
duplicates = merged_data.duplicated()
print(duplicates.value_counts())

In [ ]:
# Dropping duplicate values from the merged data 
merged_data.drop_duplicates(['User_ID','Book_Title'],inplace=True)

Here, dropping the duplicate value if they are present while merging our our dataset.

In [ ]:
# Creating pivot table
pivot_matrix=merged_data.pivot(index='Book_Title',values='Book_Rating',columns='User_ID')

Here, we have created pivot table of merged data where we passed index for Book_Title, values for Book_Rating, Column as User_ID.

In [ ]:
# pivot_matrix overview
pivot_matrix.head()

Here, most of the ratings are NaN which means there is absence of ratings.

In [ ]:
# Imputing Nan values to 0
pivot_matrix.fillna(0, inplace=True)

In [ ]:
# Viewing pivot_matrix after imputing 0
pivot_matrix.head()

In [ ]:
# csr is compressed sparse row because pivot_marrix is a sparse matrix
book_sparse = csr_matrix(pivot_matrix)

If most of the elements in the matrix are zero then the matrix is called a sparse matrix. It is wasteful to store the zero elements in the matrix since they do not affect the results of our computation.

So, here we have used csr_matrix to remove zeroes from pivot_matrix.

In [ ]:
# Modelling with KNN
Recommendation_model = NearestNeighbors(metric = 'cosine',algorithm='brute')

Here, we have used evaluation metric **cosine** which is used for determining the distance between the neighbors, in recommendation systems. 

In [ ]:
# Fitting the model 
Recommendation_model.fit(book_sparse)

In [ ]:
#Generating random book 
random_book = np.random.choice(pivot_matrix.shape[0])
print(random_book)
print(pivot_matrix.iloc[random_book,:].values.reshape(1,-1))
#Apply the k neighbors to fiited model this will create clusters
distances, indices = Recommendation_model.kneighbors(pivot_matrix.iloc[random_book,:].values.reshape(1, -1), n_neighbors = 6)
pivot_matrix.index[random_book]

Here, we choose any random book for which we need to recommend the book.

In [ ]:
#Printing the random book name and suggested books.
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(pivot_matrix.index[random_book]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, pivot_matrix.index[indices.flatten()[i]], distances.flatten()[i]))

Here, are the 5 recommended books which is been recommended by our recommendation system.

In [ ]:
#Function to get recommandation for known books from our dataset
def recommend_books(book_name):
   Book_ID = np.where(pivot_matrix.index==book_name)[0][0]
   print(Book_ID)
   distances ,suggestions = Recommendation_model.kneighbors(pivot_matrix.iloc[Book_ID,:].values.reshape(1,-1),n_neighbors = 6)
   
   for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(pivot_matrix.index[Book_ID]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, pivot_matrix.index[suggestions.flatten()[i]], distances.flatten()[i]))

Here, we have written function to recommend the model based on which book we have chosen.

In [ ]:
# Geting recommandation for book '4 Blondes'
recommend_books('4 Blondes')

Here are our 5 recommended books by our recommendation system. Here evalaution metric is near to 1 which means it is a good model.

# **Collaborative Filtering using SVD(Singular Value Decomposition)**

Singular Value Decomposition (SVD) is one of the common matrix factorization methods used in collaborative filtering, which introduces the bias information of users and items and is realized by using algebraic feature extraction. The derivative model SVD++ of SVD achieves better predictive accuracy due to the addition of implicit feedback information.

For recommendation system we have to install surprise.

In [ ]:
#Installing surprise
!pip install scikit-surprise

We, have successfully installed surprise.

Here, surprise stands for Simple Python Recommendation System Engine.

Surprise is a Python scikit building and analyzing recommender systems that deal with explicit rating data.

In [ ]:
# Importing Reader & Dataset from surprise
from surprise import Reader, Dataset
# Creating a 'Reader' object to set the limit of the ratings 
reader = Reader(rating_scale=(1, 10))
# Loading our ratings_explicit data
data = Dataset.load_from_df(merged_data[['User_ID', 'ISBN', 'Book_Rating']], reader)

In [ ]:
# Importing SVD model
from surprise import SVD, model_selection, accuracy, NMF
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV

In [ ]:
# Using  SVD algorithm
model_svd = SVD()

We have passed SVD() to model variable.

In [ ]:
# Applying cross validation
cv_results_svd = cross_validate(model_svd, data, cv=3)
pd.DataFrame(cv_results_svd).mean()

In [ ]:
# Applying NMF
model_nmf = NMF()
cv_results_nmf = cross_validate(model_nmf, data, cv=3)
pd.DataFrame(cv_results_nmf).mean()

NMF stands for Latent Semantic Analysis with the ‘Non-negative Matrix-Factorization’ method used to decompose the document-term matrix into two smaller matrices — the document-topic matrix (U) and the topic-term matrix (W) — each populated with unnormalized probabilities.

In [ ]:
#Optimising the svd model
param_grid = {'n_factors': [80,100],
              'n_epochs': [5, 20],
              'lr_all': [0.002, 0.005],
              'reg_all': [0.2, 0.4]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

Here, best Root Mean Square Error is 1.63  and best parameters which are as follows: m_factors is 80, n_epochs is 20, lr_all is 0.005, reg_all is 0.2.

In [ ]:
# Appying train_test_split
X_train, y_test = train_test_split(data, test_size=0.2)

model = SVD(n_factors=80, n_epochs=20, lr_all=0.005, reg_all=0.2)
model.fit(X_train)
predictions = model.test(y_test) 

In [ ]:
# predicting the books
pred_df = pd.DataFrame(predictions, columns=['User_ID', 'ISBN', 'Actual_Rating', 'Pred_Rating','Details'])
pred_df['Pred_Rating_Round'] = pred_df['Pred_Rating'].round()
pred_df['Abs_Error'] = abs(pred_df['Pred_Rating'] - pred_df['Actual_Rating'])
pred_df.sample(5)

Here, we can see that it has predicted well according to our Actual_rating and Predicted_rating.

Now, we will visualize absolute erroe and mean absolute error.

In [ ]:
# Grouping 
df_pred_err = pred_df.groupby('Actual_Rating')['Abs_Error'].mean().reset_index() 

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Ploting a distplot for Absolute Error
sns.distplot(pred_df['Abs_Error'], color='green', ax=ax1)
ax1.set_title('Distribution of Absolute Error')

# Ploting Barplot for Mean Absolute error
palette = sns.color_palette("RdBu", 10)
sns.barplot(x='Actual_Rating', y='Abs_Error', data=df_pred_err, palette=palette, ax=ax2)
ax2.set_title('Mean Absolute Error for rating')

plt.show()

Here, we can see that the distribution of absolute error is right-skewed, showing that the majority of errors is small: between 0 and 1. There is a long tail that indicates that there are several observations for which the absolute error was close to 10.

In [ ]:
# Merging pred_df and df_books
df_books = books.copy()
df_new = merged_data.merge(df_books[['ISBN', 'Book_Title']], on='ISBN', how='left')
df_new = df_new.merge(pred_df[['ISBN', 'User_ID', 'Pred_Rating']], on=['ISBN', 'User_ID'], how='left')

Here, we have merged our df_books and merged data with pred_df to get proper Book_Title and Book_Rating.

In [ ]:
# viewing our df_new
df_new.head()

In [ ]:
# Taking any User_ID  to train our model.
selected_user_id = 225379
df_user = df_new[df_new['User_ID']==selected_user_id]

df_user[(df_user['Pred_Rating'].isna())&(df_user['Book_Rating']>=9)].sample(5)

In [ ]:
# Test set
df_user[df_user['Pred_Rating'].notna()].sort_values('Pred_Rating', ascending=False).head(3)

In [ ]:
# Actual set
df_user[df_user['Pred_Rating'].notna()].sort_values('Book_Rating', ascending=False).head(3)

Here, we can clearly see our books are recommended well by our recommendation system. Its seems SVD be the best model among all.

# **Conclusion**



*   After doing EDA we found following conclusions:

  *   Great Expectations, Pride and Prejudice, The Night Before Christmas were the top 3 books in our books dataset.
  *   Agatha Christie, William Shakespeare, Stephen King were the top 3 authors in our books dataset.

  *   Harlequin, Silhouette, Pocket are the top 3 book publisheres in our books dataset. 
  *   Most of the users were from age group between 30-40.

  *   Most of the rating was found to be 8.


*   After applying Popularity based recommendation system The Lovely Bones: A Novel, Wild Animus, The Da Vinci Code are the 3 most popular book recommended by our recommendation system.
*   SVD is best model among all because it predicted well. 










